In [15]:
import pandas as pd
from Transformation import Transformation

In [16]:
goals = Transformation().GoalData()
games = Transformation().GameData()
players = Transformation().PlayerData()
rounds = Transformation().RoundsData()
teams = Transformation().TeamData()

In [17]:
print('Goals Has Shape: ',goals.shape)
print('Games Has Shape: ',games.shape)
print('Teams Has Shape: ',teams.shape)
print('Players Has Shape: ',players.shape)
print('Rounds Has Shape: ',rounds.shape)

Goals Has Shape:  (171, 7)
Games Has Shape:  (64, 5)
Teams Has Shape:  (222, 2)
Players Has Shape:  (123, 2)
Rounds Has Shape:  (20, 2)


* Since the dataframes are of different shape and sizes therefore we can't use merge or concatenate functions to obtain our final dataframe.
* We will be defining our own functions to create our final dataframe.

In [18]:
goals.head()

id  person_id  game_id  team_id penalty owngoal  minute
0   1          1        1      154       f       t      11
1   2          2        1      211       f       f      29
2   3          2        1      211       t       f      71
3   4          3        1      211       f       f      90
4   5          4        2      190       f       f      61

* goals dataframe consist of id's linking to various other dataframes, so  we will be using it as a reference dataframe to obtain desired data

In [19]:
#function to get player names
def get_players():
    for index in goals.index:
        person_id = goals.loc[index,'person_id']
        player = players.loc[players['id']==person_id,'name']
        goals.loc[index,'person_id'] = player.values[0]
    goals.rename(columns={'person_id':'players'},inplace=True)
get_players()
goals.head(10)

id          players  game_id  team_id penalty owngoal  minute
0   1          Marcelo        1      154       f       t      11
1   2           Neymar        1      211       f       f      29
2   3           Neymar        1      211       t       f      71
3   4            Oscar        1      211       f       f      90
4   5    Oribe Peralta        2      190       f       f      61
5   6       Ivica Olić        4      154       f       f      11
6   7     Ivan Perišić        4      154       f       f      48
7   8  Mario Mandžukić        4      154       f       f      61
8   9  Mario Mandžukić        4      154       f       f      73
9  10           Neymar        5      211       f       f      17

In [20]:
#function to get opponent team's id
def get_opponent_id():
    opponent_id = []
    for index in goals.index:
        team_id = goals.loc[index,'team_id']
        game_id = goals.loc[index,'game_id']
        team1_id = games.loc[games['id'] == game_id,'team1_id']
        team2_id = games.loc[games['id'] == game_id,'team2_id']

        opponent_id.append(team2_id.values[0])
            
    goals['opponent_id'] = opponent_id

get_opponent_id()
goals.head(10)


id          players  game_id  team_id penalty owngoal  minute  opponent_id
0   1          Marcelo        1      154       f       t      11          154
1   2           Neymar        1      211       f       f      29          154
2   3           Neymar        1      211       t       f      71          154
3   4            Oscar        1      211       f       f      90          154
4   5    Oribe Peralta        2      190       f       f      61           22
5   6       Ivica Olić        4      154       f       f      11          154
6   7     Ivan Perišić        4      154       f       f      48          154
7   8  Mario Mandžukić        4      154       f       f      61          154
8   9  Mario Mandžukić        4      154       f       f      73          154
9  10           Neymar        5      211       f       f      17          211

In [21]:
goals.loc[goals['owngoal']=='t',:]

id          players  game_id  team_id penalty owngoal  minute  \
0      1          Marcelo        1      154       f       t      11   
73    74  Noel Valladares       26      131       f       t      48   
88    89   Sead Kolašinac       31      210       f       t       3   
118  119        John Boye       42      138       f       t      31   
146  147      Joseph Yobo       53      131       f       t      90   

     opponent_id  
0            154  
73           117  
88           162  
118           18  
146           20

*owngoal column has caused presence of wrong values in team_id and opponent_id columns for certain observations*

In [22]:
goals.loc[goals['id']==1,'team_id'] = 211
goals.loc[goals['id']==74,['team_id','opponent_id']] = [117,131]
goals.loc[goals['id']==89,['team_id','opponent_id']] = [162,210]
goals.loc[goals['id']==119,['team_id','opponent_id']] = [18,138]
goals.loc[goals['id']==147,['team_id','opponent_id']] = [20,131]
goals.loc[goals['owngoal']=='t',:]

id          players  game_id  team_id penalty owngoal  minute  \
0      1          Marcelo        1      211       f       t      11   
73    74  Noel Valladares       26      117       f       t      48   
88    89   Sead Kolašinac       31      162       f       t       3   
118  119        John Boye       42       18       f       t      31   
146  147      Joseph Yobo       53       20       f       t      90   

     opponent_id  
0            154  
73           131  
88           210  
118          138  
146          131

In [23]:
#function to obtain knockout column
def get_knockout():
    knockout = []
    for index in goals.index:
        game_id = goals.loc[index,'game_id']
        knock_out = games.loc[games['id'] == game_id,'knockout']
        knockout.append(knock_out.values[0])
    goals['knockout'] = knockout

get_knockout()
goals.head(10)

id          players  game_id  team_id penalty owngoal  minute  opponent_id  \
0   1          Marcelo        1      211       f       t      11          154   
1   2           Neymar        1      211       f       f      29          154   
2   3           Neymar        1      211       t       f      71          154   
3   4            Oscar        1      211       f       f      90          154   
4   5    Oribe Peralta        2      190       f       f      61           22   
5   6       Ivica Olić        4      154       f       f      11          154   
6   7     Ivan Perišić        4      154       f       f      48          154   
7   8  Mario Mandžukić        4      154       f       f      61          154   
8   9  Mario Mandžukić        4      154       f       f      73          154   
9  10           Neymar        5      211       f       f      17          211   

  knockout  
0        f  
1        f  
2        f  
3        f  
4        f  
5        f  
6        f  
7        f  
8        f  
9        f

In [24]:
#function to get round's id
def get_round_id():
    round_id = []
    for index in goals.index:
        game_id = goals.loc[index,'game_id']
        round_ = games.loc[games['id']==game_id,'round_id']
        round_id.append(round_.values[0])
    goals['round_id'] = round_id

get_round_id()
goals.head(10)

id          players  game_id  team_id penalty owngoal  minute  opponent_id  \
0   1          Marcelo        1      211       f       t      11          154   
1   2           Neymar        1      211       f       f      29          154   
2   3           Neymar        1      211       t       f      71          154   
3   4            Oscar        1      211       f       f      90          154   
4   5    Oribe Peralta        2      190       f       f      61           22   
5   6       Ivica Olić        4      154       f       f      11          154   
6   7     Ivan Perišić        4      154       f       f      48          154   
7   8  Mario Mandžukić        4      154       f       f      61          154   
8   9  Mario Mandžukić        4      154       f       f      73          154   
9  10           Neymar        5      211       f       f      17          211   

  knockout  round_id  
0        f         1  
1        f         1  
2        f         1  
3        f         1  
4        f         2  
5        f         7  
6        f         7  
7        f         7  
8        f         7  
9        f        12

In [25]:
#funnction to get team and opponent name
def get_teamAndOpponent():
    for index in goals.index:
        team_id = goals.loc[index,'team_id']
        opponent_id = goals.loc[index,'opponent_id']
        
        team = teams.loc[teams['id']==team_id,'title']
        goals.loc[index,'team_id'] = team.values[0]
    
        opponent = teams.loc[teams['id']==opponent_id,'title']
        goals.loc[index,'opponent_id'] = opponent.values[0]
    goals.rename(columns={'team_id':'team'},inplace=True)
    goals.rename(columns={'opponent_id':'opponent'},inplace=True)

get_teamAndOpponent()
goals.head(10)

id          players  game_id     team penalty owngoal  minute  opponent  \
0   1          Marcelo        1   Brazil       f       t      11   Croatia   
1   2           Neymar        1   Brazil       f       f      29   Croatia   
2   3           Neymar        1   Brazil       t       f      71   Croatia   
3   4            Oscar        1   Brazil       f       f      90   Croatia   
4   5    Oribe Peralta        2   Mexico       f       f      61  Cameroon   
5   6       Ivica Olić        4  Croatia       f       f      11   Croatia   
6   7     Ivan Perišić        4  Croatia       f       f      48   Croatia   
7   8  Mario Mandžukić        4  Croatia       f       f      61   Croatia   
8   9  Mario Mandžukić        4  Croatia       f       f      73   Croatia   
9  10           Neymar        5   Brazil       f       f      17    Brazil   

  knockout  round_id  
0        f         1  
1        f         1  
2        f         1  
3        f         1  
4        f         2  
5        f         7  
6        f         7  
7        f         7  
8        f         7  
9        f        12

In [26]:
#functuion to get rounds
def get_rounds():
    for index in goals.index:
        round_id = goals.loc[index,'round_id']
        
        round_ = rounds.loc[rounds['id']==round_id,'title']
        goals.loc[index,'round_id'] = round_.values[0]
        
    goals.rename(columns={'round_id':'round'},inplace=True)

get_rounds()
goals.head(10)

id          players  game_id     team penalty owngoal  minute  opponent  \
0   1          Marcelo        1   Brazil       f       t      11   Croatia   
1   2           Neymar        1   Brazil       f       f      29   Croatia   
2   3           Neymar        1   Brazil       t       f      71   Croatia   
3   4            Oscar        1   Brazil       f       f      90   Croatia   
4   5    Oribe Peralta        2   Mexico       f       f      61  Cameroon   
5   6       Ivica Olić        4  Croatia       f       f      11   Croatia   
6   7     Ivan Perišić        4  Croatia       f       f      48   Croatia   
7   8  Mario Mandžukić        4  Croatia       f       f      61   Croatia   
8   9  Mario Mandžukić        4  Croatia       f       f      73   Croatia   
9  10           Neymar        5   Brazil       f       f      17    Brazil   

  knockout        round  
0        f   Matchday 1  
1        f   Matchday 1  
2        f   Matchday 1  
3        f   Matchday 1  
4        f   Matchday 2  
5        f   Matchday 7  
6        f   Matchday 7  
7        f   Matchday 7  
8        f   Matchday 7  
9        f  Matchday 12

In [27]:
#create boolean features
encode = {'f':0,'t':1}
goals['penalty']=goals['penalty'].map(encode)
goals['owngoal']=goals['owngoal'].map(encode)
goals['knockout']=goals['knockout'].map(encode)

goals.head(10)

id          players  game_id     team  penalty  owngoal  minute  opponent  \
0   1          Marcelo        1   Brazil        0        1      11   Croatia   
1   2           Neymar        1   Brazil        0        0      29   Croatia   
2   3           Neymar        1   Brazil        1        0      71   Croatia   
3   4            Oscar        1   Brazil        0        0      90   Croatia   
4   5    Oribe Peralta        2   Mexico        0        0      61  Cameroon   
5   6       Ivica Olić        4  Croatia        0        0      11   Croatia   
6   7     Ivan Perišić        4  Croatia        0        0      48   Croatia   
7   8  Mario Mandžukić        4  Croatia        0        0      61   Croatia   
8   9  Mario Mandžukić        4  Croatia        0        0      73   Croatia   
9  10           Neymar        5   Brazil        0        0      17    Brazil   

   knockout        round  
0         0   Matchday 1  
1         0   Matchday 1  
2         0   Matchday 1  
3         0   Matchday 1  
4         0   Matchday 2  
5         0   Matchday 7  
6         0   Matchday 7  
7         0   Matchday 7  
8         0   Matchday 7  
9         0  Matchday 12

Save final dataframe as a csv file

In [28]:
goals.to_csv('data.csv',index=False)